### 패키지

In [2]:
# base
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

# path
from glob import glob

# warning
import warnings
warnings.filterwarnings('ignore')

### 데이터 오류검사

In [2]:
### 오류검사 함수
def ErrorFinder(data_path) :
    path_list = sorted(glob(data_path+'/*.csv'))
    path_list

    data_list = []
    for path in path_list :
        try :
            temp_data = pd.read_csv(path, encoding = 'cp949')
            print("정상", end = " ")
        except :
            print('ERROR')
            print(path)

### 검사

In [3]:
ErrorFinder("../DATA/WEATHER/2021_신예보")

정상 정상 정상 정상 정상 정상 정상 정상 정상 정상 정상 정상 정상 정상 

In [4]:
ErrorFinder("../DATA/WEATHER/2021_구예보")

정상 정상 정상 정상 정상 정상 정상 정상 정상 정상 정상 정상 

In [5]:
ErrorFinder("../DATA/WEATHER/2020")

정상 정상 정상 정상 정상 정상 정상 정상 정상 정상 정상 정상 

In [6]:
ErrorFinder("../DATA/WEATHER/2018")

정상 정상 정상 정상 정상 정상 정상 정상 정상 정상 정상 정상 

In [7]:
ErrorFinder("../DATA/WEATHER/2019")

정상 정상 정상 정상 정상 정상 정상 정상 정상 정상 정상 정상 

In [8]:
ErrorFinder("../DATA/WEATHER/2021_6월구예보")

정상 정상 정상 정상 정상 정상 정상 정상 정상 정상 정상 정상 

### 전처리

##### 구예보 Standard function

In [57]:
### 구예보 데이터
def clean_old_forecast(data_path, value_name) :
    data = pd.read_csv(data_path)
    temp_zero = pd.DataFrame(data.columns).T
    temp_zero.columns = data.columns
    data = pd.concat([temp_zero,data],axis = 0)
    data.reset_index(drop = True, inplace = True)

    data["len"] = data[' format: day'].apply(lambda x : len(str(x)))
    split_index = data[data["len"] > 8].index
    split_index = list(split_index)
    split_index.append(len(data))

    data_list = []
    for i in range(len(split_index)-1) :
        temp = data.iloc[split_index[i]:split_index[i+1],:]
        temp.reset_index(drop = True, inplace = True)
        temp.columns = list(temp.iloc[0])
        temp.drop(0,axis = 0)
        temp.columns = ['day','hour','forecast','value','len']
        data_list.append(temp)

    start_index = data_list[0].iloc[0,3].find("S")
    data_list[0].iloc[0,0] = data_list[0].iloc[0,3][start_index:]
    data_list[0].iloc[0,1:] = np.nan

    new_data_list = []
    numbers_list = []
    
    for i in range(12) :
        try :# 12월까지 추출, 이후 중복
            string = data_list[i].iloc[0,0]
            numbers = re.sub(r'[^0-9]', '', string)
            numbers = int(numbers)
            numbers_list.append(numbers-1)
            temp = data_list[i].iloc[1:,:]
            temp.reset_index(drop = True, inplace = True)
            new_data_list.append(temp)
        except :
            pass

    for i in range(len(numbers_list)) :
        new_data_list[i]["day"] = numbers_list[i] + new_data_list[i]["day"].astype(int)

    new_data = pd.concat(new_data_list)
    new_data.reset_index(drop = True, inplace = True)

    new_data.rename(columns = {"value" : value_name},inplace = True)
    new_data  = new_data.drop(columns = "len")
    
    new_data["hour"] = new_data["hour"].astype(int)
    new_data["hour"] = new_data["hour"]/100
    
    new_data["day"] = new_data["day"].astype(str).apply(lambda x : x[0:4] + "-" +x[4:6] + "-" + x[6:8])
    new_data["hour"] = new_data["hour"].astype(int)
    
    new_data["hour"] = new_data["hour"].apply(lambda x : "0" + str(x) if (x < 10) else str(x))
    
    new_data["datetime"] = new_data["day"] + " " + new_data["hour"] + ":" + "00" + ":" + "00"
    new_data["datetime"] = pd.to_datetime(new_data["datetime"])
    new_data.drop(columns = ["day","hour"],inplace = True)
    
    try :
        new_data["forecast"] = new_data["forecast"].apply(lambda x : x.strip("+"))
    except :
        pass
    
    
    new_data["forecast"] = new_data["forecast"].astype(int)
    
    new_data["forecast_time"] = new_data["datetime"]
    
    new_data["forecast_time"] = new_data["datetime"] + pd.to_timedelta(new_data["forecast"], unit = "H") + pd.to_timedelta(9, unit="H")
    
    new_data["forecast_time"] = pd.to_datetime(new_data["forecast_time"])
    new_data["forecast"] = new_data["forecast"].astype(int)
    new_data["datetime"] = pd.to_datetime(new_data["datetime"])
    
    return new_data

### 구예보 병합
def old_concat(path) :
    dir = glob(path + "/*.csv")
    name = [i.split("/")[4].split("_")[1] for i in dir]
    old_weather = []
    for i in range(len(name)) :
        try :
            temp = clean_old_forecast(dir[i], name[i])
            old_weather.append(temp)
        except :
            print(dir[i])
        
    for i in range(1,len(old_weather)) :
        old_weather[0] = old_weather[0].merge(old_weather[i], how = "left", on = ["datetime","forecast","forecast_time"])

    return old_weather[0]

##### 신예보 function

In [59]:
def preprocessing(wind_data_path):  
    
    path_list = sorted(glob(wind_data_path+'/*.csv'))
    path_list
    
    #데이터 불러오기
    data_year_precipitation = pd.read_csv(path_list[0])
    data_year_temperature = pd.read_csv(path_list[1])
    data_year_snow = pd.read_csv(path_list[2])
    data_year_precipitationform = pd.read_csv(path_list[3])
    data_year_precipitationprob = pd.read_csv(path_list[4])    
    data_year_humidity = pd.read_csv(path_list[7])
    data_year_windspeed = pd.read_csv(path_list[11])
    data_year_winddirection = pd.read_csv(path_list[12])
    data_year_cloud = pd.read_csv(path_list[13])

    # 데이터 결합 (3시간)
    data_year = data_year_precipitation[[' format: day', 'hour', 'forecast']]
    data_year['precipitation'] = data_year_precipitation[data_year_precipitation.columns[-1]] #1시간강수량 
    data_year['temperature'] = data_year_temperature[data_year_temperature.columns[-1]] # 1시간 기온
    data_year['snow'] = data_year_snow[data_year_snow.columns[-1]] #1시간적설
    data_year['precipitationform'] = data_year_precipitationform[data_year_precipitationform.columns[-1]] #강수확률
    data_year['precipitationprob'] = data_year_precipitationprob[data_year_precipitationprob.columns[-1]] #강수형태
    data_year['humidity'] = data_year_humidity[data_year_humidity.columns[-1]] #습도
    data_year['windspeed'] = data_year_windspeed[data_year_windspeed.columns[-1]] #풍속  
    data_year['winddirection'] = data_year_winddirection[data_year_winddirection.columns[-1]] #풍향
    data_year['cloud'] = data_year_cloud[data_year_cloud.columns[-1]] #하늘상태
     
    # 일 구분 행 추출 
    day_rows = [-1]
    day_rows.extend(data_year[data_year['hour'].isna()].index)
    day_rows.append(data_year.shape[0]+1)
    
    
    # 일별 데이터 분리
    day_data = []
    for i in range(len(day_rows)-1):
        day_data.append(data_year.loc[day_rows[i]+1:day_rows[i+1]-1])    
       
    # 연월일 데이타  (첫째줄에서 첫째연월일데이타 가져오고, hour가 빈행에서 나머지 연월일 데이타 가져오기)   
    day_idx=[]
    first_day = data_year_precipitation.columns.to_list()[3].split("Start : ")[1] # 첫째줄 연월일 데이타 
    day_info = [first_day] 
    day_idx.extend(data_year[data_year['hour'].isna()].index)  
    for i in day_idx:
        day_info.append(data_year.loc[i][0].split(" Start : ")[1])    
     
    # 연월일 데이타를 date 행으로 만들고 데이타 합쳐서 dataframe만들기 
    df=None
    date_list=[]
    for i in range(len(day_data)):
        day_data[i]['date'] = day_info[i]
        df = pd.concat([df, day_data[i]],axis=0)  
    df['date'] = df['date'].str[:4] +  '-' + df['date'].str[4:6] + '-' + df['date'].str[6:8] + " " + (df['hour'].astype(int)//100).astype(str) + ':00'
    df['forecast_time'] = pd.to_datetime(df['date']) + pd.DateOffset(hours=9)    
    df = df.drop([' format: day','hour'], axis=1)

    def to_date(x):
        return pd.DateOffset(hours=x)
    df['datetime']= df['forecast_time'] + df['forecast'].map(to_date)  
    return df

# 예보시간 14시만 뺌 
def make_forecast(df):
    df = df[(df['forecast'] == 6.0) | (df['forecast'] == 7.0) | (df['forecast'] == 8.0)]
    df = df.drop(['date', 'forecast_time', 'forecast'],axis=1) 
    df= df.reset_index(drop=True)
    return df

### 데이터 불러오기

In [80]:
### LOAD DATA
old_2018 = old_concat("../DATA/WEATHER/2018")
old_2019 = old_concat("../DATA/WEATHER/2019")
old_2020 = old_concat("../DATA/WEATHER/2020")
old_2021 = old_concat("../DATA/WEATHER/2021_구예보")

new_2021 = make_forecast(preprocessing("../DATA/WEATHER/2021_신예보"))

In [81]:
old_2018 = old_2018[old_2018["forecast"] == 4]
old_2019 = old_2019[old_2019["forecast"] == 4]
old_2020 = old_2020[old_2020["forecast"] == 4]
old_2021 = old_2021[old_2021["forecast"] == 4]

In [82]:
### 시점 고정
old_2018.drop(columns = ["datetime", "forecast"], inplace = True)
old_2018.rename(columns = {"forecast_time" : "datetime"}, inplace = True)

old_2019.drop(columns = ["datetime", "forecast"], inplace = True)
old_2019.rename(columns = {"forecast_time" : "datetime"}, inplace = True)

old_2020.drop(columns = ["datetime", "forecast"], inplace = True)
old_2020.rename(columns = {"forecast_time" : "datetime"}, inplace = True)

old_2021.drop(columns = ["datetime", "forecast"], inplace = True)
old_2021.rename(columns = {"forecast_time" : "datetime"}, inplace = True)

In [83]:
old_weather = pd.concat([old_2018,old_2019,old_2020,old_2021])
old_weather = old_weather[['datetime', '3시간기온', '습도', '풍향']]
old_weather.rename(columns = {"3시간기온" : "기온"}, inplace = True)

In [84]:
old_df= pd.DataFrame(columns=['datetime'])
old_df['datetime'] = pd.date_range(start='2019-01-01 00:00:00', end='2021-06-30 08:00:00', freq='H')
old_weather = pd.merge(old_df,old_weather, on ='datetime', how='outer')
old_weather['기온'] = old_weather['기온'].astype('float').interpolate()
old_weather['습도'] = old_weather['습도'].astype('float').interpolate()
old_weather['풍향'] = old_weather['풍향'].astype('float').interpolate()

In [85]:
old_weather = old_weather.loc[(old_weather['datetime'] < '2021-06-30 08:00:00') & (old_weather['datetime'] >'2019')]
old_weather

,datetime,기온,습도,풍향
1,2019-01-01 01:00:00,-2.000000,35.000000,286.000000
2,2019-01-01 02:00:00,-3.000000,35.000000,288.000000
3,2019-01-01 03:00:00,-4.000000,35.000000,290.000000
4,2019-01-01 04:00:00,-4.000000,36.666667,282.666667
5,2019-01-01 05:00:00,-4.000000,38.333333,275.333333
...,...,...,...,...
21867,2021-06-30 03:00:00,19.000000,95.000000,202.000000
21868,2021-06-30 04:00:00,19.333333,95.000000,204.000000
21869,2021-06-30 05:00:00,19.666667,95.000000,206.000000
21870,2021-06-30 06:00:00,20.000000,95.000000,208.000000


In [86]:
new_2021["datetime"] = pd.to_datetime(new_2021["datetime"])
new_2021.rename(columns = {"temperature" : "기온", "humidity" : "습도", "windspeed" : "풍속", "winddirection" : "풍향"}, inplace = True)
new_2021 = new_2021[['datetime', '기온', '습도', '풍향']]
new_2021 = new_2021.loc[new_2021['datetime'] < '2022']

In [87]:
weather = pd.concat([old_weather, new_2021], axis = 0)
weather.reset_index(drop=True, inplace = True)
weather

,datetime,기온,습도,풍향
0,2019-01-01 01:00:00,-2.0,35.000000,286.000000
1,2019-01-01 02:00:00,-3.0,35.000000,288.000000
2,2019-01-01 03:00:00,-4.0,35.000000,290.000000
3,2019-01-01 04:00:00,-4.0,36.666667,282.666667
4,2019-01-01 05:00:00,-4.0,38.333333,275.333333
...,...,...,...,...
26298,2021-12-31 19:00:00,-2.0,20.000000,290.000000
26299,2021-12-31 20:00:00,-3.0,20.000000,288.000000
26300,2021-12-31 21:00:00,-3.0,25.000000,284.000000
26301,2021-12-31 22:00:00,-3.0,25.000000,282.000000


### 정리

In [88]:
weather.to_csv("../DATA/WEATHER_Copy1.csv", index = False)